In [1]:
import os
SNAPPY_notebook_path = os.path.join(os.path.abspath(""), "bench.ipynb")

In [2]:
import time
SNAPPY_start_time = time.perf_counter_ns()

In [3]:
%load_ext autotime

time: 40.3 µs (started: 2024-05-07 17:19:42 -04:00)


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import os
# STEFANOS: Conditionally import Modin Pandas
import pandas as pd

# Input data files are available in the read-only "input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(os.path.dirname('') + '/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

time: 227 ms (started: 2024-05-07 17:19:42 -04:00)


In [5]:
def hash_anything(obj):
    import pandas.util
    if isinstance(obj, pd.DataFrame):
        return pandas.util.hash_pandas_object(obj, index=False).to_numpy().data
    elif isinstance(obj, np.ndarray):
        return obj.data
    elif isinstance(obj, list):
        return str(obj).encode()
    else:
        return str(obj).encode()


def hash_dataframe(df):
    import xxhash
    h = xxhash.xxh64()
    for column in df.round(6).columns:
        h.update(hash_anything(df[column]))
    return h.digest()


time: 474 µs (started: 2024-05-07 17:19:42 -04:00)


In [6]:
df = pd.read_table('input/imdb-official-movies-dataset/title-ratings.tsv', low_memory=False)
df2 = pd.read_table('input/imdb-official-movies-dataset/title-metadata.tsv',low_memory=False)
if "IREWR_LESS_REPLICATION" in os.environ and os.environ["IREWR_LESS_REPLICATION"] == "True":
    df2 = df2[:1_500_000]

time: 382 ms (started: 2024-05-07 17:19:42 -04:00)


In [7]:
print(hash_dataframe(df2))

b'\x81\x8e\xdeuP\xbf6\xef'
time: 6.67 ms (started: 2024-05-07 17:19:43 -04:00)


In [8]:
# Merging Rating and MetaData datsets into single dataframe w.r.t common column
main_df = pd.merge(df,df2,on='tconst')
main_df.head()

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt1006009,8.1,7,tvSeries,Title 10219,Original Title 92274,1,1939,\N,51,Action
1,tt1013711,7.3,44,tvEpisode,Title 95852,Original Title 48663,0,2011,\N,182,Comedy
2,tt1014752,7.9,69,movie,Title 22376,Original Title 31800,0,2000,\N,171,Comedy
3,tt1015625,7.2,129,tvEpisode,Title 94679,Original Title 93046,1,1905,\N,172,Comedy
4,tt1029262,6.5,19,tvEpisode,Title 61657,Original Title 80940,1,1914,\N,99,Comedy


time: 233 ms (started: 2024-05-07 17:19:43 -04:00)


**DATA Cleaning:**

In [9]:
# creating a copy 
imdb_df = main_df.copy()

time: 656 µs (started: 2024-05-07 17:19:43 -04:00)


**Currently, 'runtimeMinutes' column is in Minutes form thus we will convert it into Days:HH:MM:SS Format for better display of runtime information**

In [10]:
imdb_df['runtimeMinutes'].value_counts()

runtimeMinutes
280    7
74     6
17     6
283    6
4      6
      ..
262    1
12     1
10     1
258    1
268    1
Name: count, Length: 258, dtype: int64

time: 1.77 ms (started: 2024-05-07 17:19:43 -04:00)


In [11]:
# Replace object type string '\N' with NaN First
imdb_df['runtime_delta'] = pd.to_numeric(imdb_df['runtimeMinutes'], errors='coerce', downcast="integer")

time: 3.48 ms (started: 2024-05-07 17:19:43 -04:00)


In [12]:
# Using 'pd.to_timedelta' to convert minutes into desired format
imdb_df['runtime_delta'] = pd.to_timedelta(imdb_df['runtime_delta'], unit='m')

time: 431 µs (started: 2024-05-07 17:19:43 -04:00)


**Similarly, The 'isAdult' column should contain only two values, '0: non-adult title, 1: adult title' but here As we can see dataFrame column consist of mixed data values like string, bool and int. Thus, Lets sort and restructure this column into a Bool data type.**

In [13]:
imdb_df['isAdult'].unique()

array([1, 0])

time: 1.1 ms (started: 2024-05-07 17:19:43 -04:00)


In [14]:
# Replace string and int by boolean
# we will leave out '2014' and '2020' as it is as there`s no way to confirm If they Adult or not not
imdb_df['isAdult'] = imdb_df['isAdult'].map({'0': 'Non_Adult_Title', '1': 'Adult_Title'})

time: 756 µs (started: 2024-05-07 17:19:43 -04:00)


In [15]:
imdb_df.isAdult = imdb_df.isAdult.fillna('Unrated')
imdb_df['isAdult'].unique()

array(['Unrated'], dtype=object)

time: 1.37 ms (started: 2024-05-07 17:19:43 -04:00)


**Creating a new column to provide information of show run timeline in better way by joinning 'startYear' and 'endYear' columns into one**

In [16]:
# First, we should replace '\N' values to nan so next step
imdb_df['startYear'] = imdb_df['startYear'].replace('\\N', np.NaN)
imdb_df['endYear'] = imdb_df['endYear'].replace('\\N', np.NaN)

time: 849 µs (started: 2024-05-07 17:19:43 -04:00)


/tmp/ipykernel_19583/2240090600.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  imdb_df['endYear'] = imdb_df['endYear'].replace('\\N', np.NaN)


In [17]:
# Now joinning them
# STEFANOS-DISABLE-FOR-MODIN
### ORIGINAL ###
# imdb_df['premiere_timeline'] = imdb_df[['startYear','endYear']].stack().groupby(level=0).agg('-'.join)
### COMPATIBLE WITH MODIN ###
# I tried a bunch of things, here's one:
#   imdb_df['premiere_timeline'] = imdb_df[['startYear','endYear']].stack().groupby(level=0).apply('-'.join)
# They don't seem to work. The problem seems to be with groupby(level=0). Generally, I'm not sure that Modin can
# groupby MultiIndex's.
# So, below I do the computation up to the stacking, just so that we can keep as much of it as possible (in case
# Modin, or some other system that uses the same benchmarks, can optimize it; we don't want to deprive this
# opportunity). Unfortunately, we cannot assign the result of stacking to premiere_timeline as they don't match.
# I just assign a dumb value to premiere_timeline so that it exists in the DF. It's not used later.

_ = imdb_df[['startYear','endYear']].stack()
imdb_df['premiere_timeline'] = 0

time: 1.73 ms (started: 2024-05-07 17:19:43 -04:00)


**The data cleaning has been completed. However, Lets drop unneccessory columns from this dataframe for furthur analysis:**

In [18]:
# Drop unneccessory columns from dataframe
columns_to_keep = ['tconst','averageRating','numVotes','titleType','primaryTitle',
                   'isAdult','premiere_timeline','runtime_delta']

imdb_df = imdb_df[columns_to_keep]

# Also renaming them with approriate name
renamed_cols = {'tconst':'IMDB_ID','averageRating':'Avg_Rating','numVotes':'Total_Votes',
                'titleType':'Title_Category','primaryTitle':'Title_Name','isAdult':'IN-18+'
                ,'premiere_timeline':'Air_time','runtime_delta':'Title_Runtime_Length'}

imdb_df.rename(columns = renamed_cols,inplace=True)
imdb_df.head()

,IMDB_ID,Avg_Rating,Total_Votes,Title_Category,Title_Name,IN-18+,Air_time,Title_Runtime_Length
0,tt1006009,8.1,7,tvSeries,Title 10219,Unrated,0,0 days 00:51:00
1,tt1013711,7.3,44,tvEpisode,Title 95852,Unrated,0,0 days 03:02:00
2,tt1014752,7.9,69,movie,Title 22376,Unrated,0,0 days 02:51:00
3,tt1015625,7.2,129,tvEpisode,Title 94679,Unrated,0,0 days 02:52:00
4,tt1029262,6.5,19,tvEpisode,Title 61657,Unrated,0,0 days 01:39:00


time: 5.12 ms (started: 2024-05-07 17:19:43 -04:00)


**Top 10 highest voted and rated Titles across all categories**

In [19]:
top_10_highest = imdb_df.groupby(['Title_Name'])[
    ['Avg_Rating','Total_Votes']].max().sort_values(by=['Avg_Rating','Total_Votes'
                                                       ], ascending = False).reset_index().head(10)

top_10_highest

,Title_Name,Avg_Rating,Total_Votes
0,Title 19158,10.0,35
1,Title 86824,10.0,28
2,Title 90504,10.0,12
3,Title 74777,10.0,5
4,Title 25000,9.8,8
5,Title 94895,9.7,7
6,Title 76819,9.7,6
7,Title 23156,9.6,27
8,Title 22126,9.6,11
9,Title 80818,9.6,5


time: 6.44 ms (started: 2024-05-07 17:19:43 -04:00)


**Many Many other titles received the Perfect Rating of 10 across all Category but My personal favorite movie of all time, 'Fight Club' tops the table with Perfect Rating and highest Individual votes received while 'Gladiator' and 'Die Hard' takes the 2nd and 3rd spot respectivly on IMDB table.**

**Average votes and rating given to all categories**

In [20]:
highest_rated_type = imdb_df.groupby(['Title_Category'])[['Avg_Rating','Total_Votes']].mean(
).sort_values(by=['Avg_Rating','Total_Votes'], ascending = False).reset_index().round(decimals = 1)

highest_rated_type

,Title_Category,Avg_Rating,Total_Votes
0,video,7.1,785.5
1,tvEpisode,7.0,206.5
2,short,7.0,375.9
3,movie,6.9,224.9
4,tvSeries,6.9,459.4


time: 4.58 ms (started: 2024-05-07 17:19:43 -04:00)


**Seems like 'tvEpisode' Category has received the best avg. Rating of 7.4/title when compaired to all other title types but received very less avg. votes per title. On the other hand, 'Movie' Category tops the chart as they received highest 3532.7 votes/titles on IMDB.**

**Category_wise longest running titles as per IMDB_database**

In [21]:
longest_runtime = imdb_df.loc[imdb_df.groupby(['Title_Category'], 
                                              sort=False)['Title_Runtime_Length'].idxmax()
                             ][['Title_Category', 'Title_Name', 'Title_Runtime_Length']]

longest_runtime.sort_values(by='Title_Runtime_Length', ascending = False).reset_index(drop=True)

,Title_Category,Title_Name,Title_Runtime_Length
0,tvSeries,Title 1132,0 days 05:00:00
1,movie,Title 82850,0 days 05:00:00
2,tvEpisode,Title 28597,0 days 04:59:00
3,short,Title 94289,0 days 04:58:00
4,video,Title 59141,0 days 04:57:00


time: 4.76 ms (started: 2024-05-07 17:19:43 -04:00)


**In our analysis, Movie named 'Logistics' has the longest run-time across all categories, at 35 Days and 17 Hours. While, 'The Longest Video on YouTube: 596.5 Hours' is in the second place.**

**Now, Lets check Category_wise shortest running titles as per IMDB_database**

In [22]:
shortest_runtime = imdb_df.loc[imdb_df.groupby(['Title_Category'], 
                                              sort=False)['Title_Runtime_Length'].idxmin()
                             ][['Title_Category', 'Title_Name', 'Title_Runtime_Length']]

shortest_runtime.reset_index(drop=True)

,Title_Category,Title_Name,Title_Runtime_Length
0,tvSeries,Title 19531,0 days 00:01:00
1,tvEpisode,Title 50970,0 days 00:01:00
2,movie,Title 42689,0 days 00:04:00
3,video,Title 56843,0 days 00:03:00
4,short,Title 4211,0 days 00:04:00


time: 3.74 ms (started: 2024-05-07 17:19:43 -04:00)


**Short category film 'Awakening of Rip' holds the top place for being the shortest length title in IMDB_database.**

**Now, Lets shed some light on Average Ratings of Non_Adult Rated Titles vs Adult Rated Titles**

In [23]:
wanna_adult_or_not = imdb_df.groupby(['Title_Category','IN-18+']
                                    )['Avg_Rating'].agg('mean').round(1).unstack(fill_value= 0) 
wanna_adult_or_not.reset_index(inplace = True)
##### Visualise the chart in stack manner of bar type

# STEFANOS: Disable plotting
# plt.rcParams['figure.figsize'] = [15, 8]
# wanna_adult_or_not.plot(x='Title_Category', kind='bar', stacked=True, 
#                         title='Average Ratings of Non_Adult Rated Titles vs Adult Rated Titles')
# plt.xlabel('Title Categories')
# plt.ylabel('Average Ratings')
# plt.show()

time: 1.74 ms (started: 2024-05-07 17:19:43 -04:00)


**'TvEpisode' category has received the highest Avg. Ratings in both Adult and Non-Adult rated title categories, However they there also the only title category among all whose some of the films/shows went Un-Rated.**

**Here ends this project, where we dig deep into the dataframe. First, We cleaned it and then sorted the data structure making a display of meaningful data. If you want to give any suggestion and point out mistakes please feel free to contact. Thanks!**

In [24]:
SNAPPY_end_time = time.perf_counter_ns()
print("Total elapsed time:", (SNAPPY_end_time - SNAPPY_start_time) / (10 ** 9))

Total elapsed time: 0.967675991
time: 231 µs (started: 2024-05-07 17:19:43 -04:00)
